In [ ]:
import cv2
import numpy as np
import gdown
from matplotlib import pyplot as plt

In [ ]:
gdown.download('https://www.intermedia.ru/img/news_x400/386271.jpg', None, quiet=True)

img = cv2.imread('./386271.jpg')
img2 = img.copy()

In [ ]:
classifier_face = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
classifier_eye = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = classifier_face.detectMultiScale(gray, scaleFactor=2, minNeighbors=2, minSize=(50, 50))

def blur_face(face, mask):
    blurred_face = cv2.GaussianBlur(face, (99, 99), 0)
    return cv2.bitwise_and(blurred_face, blurred_face, mask=mask)

for (x, y, w, h) in faces:
    face = img[y:y+h, x:x+w]
    face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    eyes = classifier_eye.detectMultiScale(face_gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

    center = (w // 2, h // 2)
    axes = (w // 2, h // 3)
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.ellipse(mask, center, axes, 90, 0, 360, 255, -1)

    # Вырезаем глаза из маски
    for (ex, ey, ew, eh) in eyes:
        center_x = ex + ew // 2
        center_y = ey + eh // 2
        radius = max(ew, eh) // 2
        cv2.circle(mask, (center_x, center_y), radius, 0, -1)

    # Применяем размытие только к области маски
    blurred_face = blur_face(face, mask)
    face_with_eyes = cv2.bitwise_and(face, face, mask=cv2.bitwise_not(mask))
    combined_face = cv2.add(face_with_eyes, blurred_face)
    img2[y:y+h, x:x+w] = combined_face


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax1.set_title('Исходное изображение')
ax1.axis('off')

ax2.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax2.set_title('Изображение после обработки')
ax2.axis('off')

plt.show()